# Kpf-Bert를 이용한 뉴스기사 3줄요약 서비스  
전체기사에서 중요한 순서대로 상위 3개의 문장을 추출해서 제시하는 기사 요약 서비스이다.  
  
pytorch-lightning을 이용하여 전체프로세서를 작성하였다.  
  
BERT를 이용한 SUMMARY 관련 논문 및 nlpyang의 PreSumm 소스를 참조하였다.

In [1]:
import math
import pandas as pd
import numpy as np
import os

from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import torch.optim as optim

from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

from torch.nn.init import xavier_uniform_

import pytorch_lightning as pl
from torchmetrics.functional import accuracy
#from pytorch_lightning.metrics.functional import accuracy, f1, auroc
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

import kss

%matplotlib inline
%config InlineBackend.figure_format='retina'

RANDOM_SEED = 42

sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

pl.seed_everything(RANDOM_SEED)

# GPU 사용 설정
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "0"  # Set the GPU to use

/home/lhch9550/anaconda3/lib/python3.8/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/lhch9550/anaconda3/lib/python3.8/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/lhch9550/anaconda3/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(
Seed set to 42


In [2]:
MAX_TOKEN_COUNT = 512
N_EPOCHS = 1
BATCH_SIZE = 8

# data
대신 AI - HUB 에서 한국어 요약 데이터셋을 공개하여 이를 활용하였다.

In [3]:
import json

DATA_TRAIN_PATH = '/home/lhch9550/공모전/train_original.json'

# JSON 파일 로드 및 확인
try:
    with open(DATA_TRAIN_PATH, 'r', encoding='utf-8') as f:
        data = json.load(f)
    print("JSON 파일이 올바르게 로드되었습니다.")
except json.JSONDecodeError as e:
    print("JSON 파일에 문제가 있습니다:", e)

JSON 파일이 올바르게 로드되었습니다.


In [6]:
def preprocess_data(data):
    outs = []
    for doc in data['documents']:
        line = []
        line.append(doc['media_name'])
        line.append(doc['id'])
        para = []
        for sent in doc['text']:
            for s in sent:
                para.append(s['sentence'])
        line.append(para)
        line.append(doc['abstractive'][0])
        line.append(doc['extractive'])
        a = doc['extractive']
        if a[0] == None or a[1] == None or a[2] == None:
            continue
        outs.append(line)

    outs_df = pd.DataFrame(outs)
    outs_df.columns = ['media', 'id', 'article_original', 'abstractive', 'extractive']
    return outs_df

In [7]:
df = pd.read_json(DATA_TRAIN_PATH)
df = df.dropna()
len(df)

243983

In [8]:
DATA_TEST_PATH = '/home/lhch9550/공모전/valid_original.json'
test_df = pd.read_json(DATA_TEST_PATH)
test_df = test_df.dropna()
len(test_df)

30122

In [9]:
train_df, val_df = train_test_split(df, test_size=0.1)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
train_df.shape, val_df.shape, test_df.shape

((219584, 3), (24399, 3), (30122, 3))

In [12]:
train_df = preprocess_data(train_df)
train_df.head(1)

,media,id,article_original,abstractive,extractive
0,매일경제,354255482,"[채권시장에서 20년 만기 국고채 금리가 10년 만기 금리 아래로 떨어졌다., 4개...",채권시장에서 20년물 금리가 10년물보다 낮은 '장기.초장기 채권금리간 역전' 현상...,"[6, 2, 12]"


In [13]:
i = 8
print('===== 본    문 =====')
for idx, str in enumerate(train_df['article_original'][i]):
    print(idx,':',str)
print('===== 요약정답 =====')
print(train_df['extractive'][i])
print('===== 추출본문 =====')
print('1 :', train_df['article_original'][i][train_df['extractive'][i][0]])
print('2 :', train_df['article_original'][i][train_df['extractive'][i][1]])
print('3 :', train_df['article_original'][i][train_df['extractive'][i][2]])
print('===== 생성본문 =====')
print(train_df['abstractive'][i])

===== 본    문 =====
0 : 이용섭 시장, 유엔인권최고대표사무소 길모어 부대표 면담서 제안
1 : 오치남 기자
2 : 광주시·유엔, 2020년 세계인권도시포럼 공동 개최 추진
3 : 이용섭 시장, 유엔인권최고대표사무소 길모어 부대표 면담서 제안
4 : 亞국가 국제인권교육 진행…유엔과 인권증진 협력 강화도
5 : 이용섭 광주광역시장이 17일(현지시간) 오전 스위스 제네바 유엔인권최고대표사무소에서 열린 ‘유엔·지방정부연합 인권협의회의’에 참석, ‘인권도시 광주, 그리고 지구적 협력’을 주제로 개회연설을 하고 있다.
6 : /광주시 제공
7 : 세계인권도시로 자리잡은 광주광역시가 유엔과 2020세계인권도시포럼 공동개최를 추진하기로 했다.
8 : 또 유엔과 인권증진 협력을 강화하고 아시아 국가를 대상으로 국제인권교육도 진행하기로 했다.
9 : 이용섭 광주시장은 17일(현지시간) 스위스 제네바에서 유엔인권최고대표사무소(UN OHCHR) 케이트 길모어 부대표와 면담을 갖고 5·18민주화운동 40주년에 맞춰 유엔인권최고대표부와 내년 5월 세계인권도시포럼 공동개최를 추진키로 했다.
10 : 이 시장은 이와 관련한 내용을 길모어 부대표에 전달하고 미첼 바첼렛 대표(전 칠레 대통령)를 포럼에 공식 초청했다.
11 : 이에 대해 길모어 부대표는 “인권의 가치와 비전을 함께 공유하고 있는 유엔과 광주가 성공적인 협업모델을 만들 수 있도록 포럼 공동개최에 대해 논의를 진행하겠다”고 말했다.
12 : 이와 함께 광주시는 2020년부터 진행하는 국제인권교육에 유엔의 교육콘텐츠와 전문강사, 노하우도 공유하기로 했다.
13 : 국제인권교육은 아시아 국가들의 인권정책 전문성 강화와 국민들의 인권증진을 위해 광주시와 코이카(KOICA)가 공동으로 실시하는 것으로, 유엔은 한국사무소를 통해 광주시와 구체적인 협력 내용을 협의하기로 했다.
14 : 이용섭 광주광역시장이 17일(현지시간) 오전 스위스 제네바 유엔인권최고대표사무소에서 열린 ‘유엔·지방정부연합 인권협의회의’에 참석한 후 케

In [14]:
test_df = preprocess_data(test_df)
test_df.head(1)

,media,id,article_original,abstractive,extractive
0,한국경제,340626877,"[[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, ...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...,"[0, 1, 3]"


In [15]:
train_df.shape, test_df.shape, val_df.shape

((219578, 5), (30121, 5), (24399, 3))

# preprocess  
  
AI-HUB 문서요약 데이터셋 기준 전처리  
-> 기존 Bflysoft-뉴스기사 데이터셋에 맞춰 변환  
위 데이터를 구하지 못해서 다른 데이터를 변환한 것이다.  
사실 거칠 필요가 없는 과정이 되어버렸다.

In [16]:
def preprocess_data(data):
    outs = []
    for doc in data['documents']:
        line = []
        line.append(doc['media_name'])
        line.append(doc['id'])
        para = []
        for sent in doc['text']:
            for s in sent:
                para.append(s['sentence'])
        line.append(para)
        line.append(doc['abstractive'][0])
        line.append(doc['extractive'])
        a = doc['extractive']
        if a[0] == None or a[1] == None or a[2] == None:
            continue
        outs.append(line)

    outs_df = pd.DataFrame(outs)
    outs_df.columns = ['media', 'id', 'article_original', 'abstractive', 'extractive']
    return outs_df

In [17]:
train_df

,media,id,article_original,abstractive,extractive
0,매일경제,354255482,"[채권시장에서 20년 만기 국고채 금리가 10년 만기 금리 아래로 떨어졌다., 4개...",채권시장에서 20년물 금리가 10년물보다 낮은 '장기.초장기 채권금리간 역전' 현상...,"[6, 2, 12]"
1,매일경제,351711827,"[중소 증권사 오너들이 경영 보폭을 넓히고 있다., 캐피털 진출과 광고 확대, 대학...","7일 금융투자업계에 따르면 , 금융위원회는 리딩투자증권의 캐피탈 출자 승인 안건을 ...","[2, 1, 0]"
2,제주일보,356271062,"[제주도, 9월 30일까지...수급조절 유통처리대책 마련 위해 신고 당부, 제주지역...",13일 제주특별자치도는 다음달 30일까지 채소류 재배면적 신고를 접수한다고 밝혔는데...,"[2, 3, 7]"
3,아주경제,358585637,"[서울시가 강남북 균형발전을 위해 2024년까지 인재개발원, 서울연구원, 서울주택도...",서울시가 강남북 균형발전을 위해 2024년까지 3개의 공공기관을 강북으로 이전하겠다...,"[0, 1, 7]"
4,제민일보,337240119,"[제주도감사위, 12일 제주시·서귀포시교육지원 감사 결과 발표, 제주시교육지원청이 ...",제주도감사위원회는 종합감사를 실시하여 제주시교육지원청이 성과 상여금을 잘못 지급하고...,"[1, 2, 3]"
...,...,...,...,...,...
219573,전북도민일보,342904723,[전북도는 24일 고사동 객리단길 내 전주영화도서관&카페에서 '제1회 도란도란 토크...,"전북도는, 도지사 공약사업 실천 및 청년들의 아이디어를 도정에 접목시키기 위해, 2...","[0, 1, 4]"
219574,전라일보,353473510,"[홍민희 기자l minihong2503@naver.com, 우수한 연구원들의 정주여...",전북테크노파크가 오는 24일까지 R&D기관 우수연구원 유치를 위한 주거비 지원사업을...,"[2, 3, 4]"
219575,중부매일,339576576,[불법행위 '택시기사 삼진아웃제' 6월 시행[중부매일 이민우 기자] 지난달 충북지역...,"청주시는 오는 6월부터 승차거부, 부당요금 징수 등 불법행위를 최근 2년 이내 3회...","[2, 4, 7]"
219576,충청일보,332010887,"[도교육청, 오는 3월 예정, 내달 15일까지 단원 모집, [충청일보 이정규기자] ...","충북예술교육을 이끌어 갈 충북청소년국악관현악단이 오는 3월 창단을 목표로 도내 초,...","[3, 4, 10]"


In [18]:
test_df

,media,id,article_original,abstractive,extractive
0,한국경제,340626877,"[[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, ...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...,"[0, 1, 3]"
1,한국경제,340626896,"[] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다., 바른미래당이 8일 ...",8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...,"[2, 1, 6]"
2,한국경제,340626904,"[[ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀., 지난 3일 한국이 세계 최초...",지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...,"[1, 5, 8]"
3,한국경제,340627450,[] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...,"[0, 1, 2]"
4,한국경제,340627465,"[[ 임근호 기자 ] ""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를...",주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...,"[1, 3, 4]"
...,...,...,...,...,...
30116,헤럴드경제,350851474,[영주시는 이억만리에서 건너온 계절근로자들의 향수를 달래고 안정된 한국생활 적응을 ...,"영주시는 외국인 근로자들의 향수를 달래고, 안정적인 한국생활 적응을 지원하기 위해 ...","[0, 1, 2]"
30117,헤럴드경제,350851925,[여름 방학을 맞아 전국의 국립과학관에서 달 탐사 50주년과 국제천문연맹(IAU) ...,달 탐사 50주년과 국제천문연맹(IAU)설립 100주년 기념하는 특별전시가 전국의 ...,"[0, 1, 12]"
30118,헤럴드경제,350854748,[영주문경예천 당원협의회 시국강연 및 당원교육에 참석차 영주를 방문한 황교안(오른쪽...,황교안 대표는 지난 29일 자유한국당 대표로 경북 영주문경예청 당원협의회 당원교육에...,"[2, 0, 1]"
30119,헤럴드경제,350857648,[경북예천군은 장기적인 국내 경기침체가 지속됨에 따라 사회적경제 기업 육성 등을 통...,경북예천군은 사회적경제 기업 육성 등을 통한 일자리 창출에 행정동력을 집중한 결과 ...,"[0, 3, 4]"


In [19]:
i = 8
print('===== 본    문 =====')
for idx, str in enumerate(train_df['article_original'][i]):
    print(idx,':',str)
print('===== 요약정답 =====')
print(train_df['extractive'][i])
print('===== 추출본문 =====')
print('1 :', train_df['article_original'][i][train_df['extractive'][i][0]])
print('2 :', train_df['article_original'][i][train_df['extractive'][i][1]])
print('3 :', train_df['article_original'][i][train_df['extractive'][i][2]])
print('===== 생성본문 =====')
print(train_df['abstractive'][i])

===== 본    문 =====
0 : 이용섭 시장, 유엔인권최고대표사무소 길모어 부대표 면담서 제안
1 : 오치남 기자
2 : 광주시·유엔, 2020년 세계인권도시포럼 공동 개최 추진
3 : 이용섭 시장, 유엔인권최고대표사무소 길모어 부대표 면담서 제안
4 : 亞국가 국제인권교육 진행…유엔과 인권증진 협력 강화도
5 : 이용섭 광주광역시장이 17일(현지시간) 오전 스위스 제네바 유엔인권최고대표사무소에서 열린 ‘유엔·지방정부연합 인권협의회의’에 참석, ‘인권도시 광주, 그리고 지구적 협력’을 주제로 개회연설을 하고 있다.
6 : /광주시 제공
7 : 세계인권도시로 자리잡은 광주광역시가 유엔과 2020세계인권도시포럼 공동개최를 추진하기로 했다.
8 : 또 유엔과 인권증진 협력을 강화하고 아시아 국가를 대상으로 국제인권교육도 진행하기로 했다.
9 : 이용섭 광주시장은 17일(현지시간) 스위스 제네바에서 유엔인권최고대표사무소(UN OHCHR) 케이트 길모어 부대표와 면담을 갖고 5·18민주화운동 40주년에 맞춰 유엔인권최고대표부와 내년 5월 세계인권도시포럼 공동개최를 추진키로 했다.
10 : 이 시장은 이와 관련한 내용을 길모어 부대표에 전달하고 미첼 바첼렛 대표(전 칠레 대통령)를 포럼에 공식 초청했다.
11 : 이에 대해 길모어 부대표는 “인권의 가치와 비전을 함께 공유하고 있는 유엔과 광주가 성공적인 협업모델을 만들 수 있도록 포럼 공동개최에 대해 논의를 진행하겠다”고 말했다.
12 : 이와 함께 광주시는 2020년부터 진행하는 국제인권교육에 유엔의 교육콘텐츠와 전문강사, 노하우도 공유하기로 했다.
13 : 국제인권교육은 아시아 국가들의 인권정책 전문성 강화와 국민들의 인권증진을 위해 광주시와 코이카(KOICA)가 공동으로 실시하는 것으로, 유엔은 한국사무소를 통해 광주시와 구체적인 협력 내용을 협의하기로 했다.
14 : 이용섭 광주광역시장이 17일(현지시간) 오전 스위스 제네바 유엔인권최고대표사무소에서 열린 ‘유엔·지방정부연합 인권협의회의’에 참석한 후 케

# tokenizer
kpfBERT 토크나이저를 바로 쓴다.
kpfBERT 토크나이저는 형태소와 유사하게 잘 토크나이징을 하게 설계되어 있다.

In [20]:
BERT_MODEL_NAME = "/home/lhch9550/공모전" # kpf-BERT 경로 입력
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

# dataset
bert에서 여러문장을 입력하기 위해 presumm 에서 제안한 형식으로 인코딩 한다.  
  
token embedding : < CLS > 문장 < SEP > 문장 < SEP > 문장 ... 문장 < SEP >  
interval segment : 0 , 0 , 0 , 1 , 1 , 0 , 0 , ... 1 , 1  
position embedding : 1 , 1 , 1 , 1 , 1 , 1 , 1 , ... 1 , 1  
  

In [21]:
class SummDataset(Dataset): # 데이터셋 presumm 방식 인코딩

    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: BertTokenizer,
        max_token_len: int = 512
    ):
        self.tokenizer = tokenizer
        self.data = data
        self.max_token_len = max_token_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]

        tokenlist = []
        for sent in data_row.article_original:
            tokenlist.append(tokenizer(
                text = sent,
                add_special_tokens = True)) #, # Add '[CLS]' and '[SEP]'

        src = [] # 토크나이징 된 전체 문단
        labels = []  # 요약문에 해당하면 1, 아니면 0으로 문장수 만큼 생성
        segs = []  #각 토큰에 대해 홀수번째 문장이면 0, 짝수번째 문장이면 1을 매핑
        clss = []  #[CLS]토큰의 포지션값을 지정

        odd = 0
        for tkns in tokenlist:
            if odd > 1 : odd = 0
            clss = clss + [len(src)]
            src = src + tkns['input_ids']
            segs = segs + [odd] * len(tkns['input_ids'])
            if tokenlist.index(tkns) in data_row.extractive :
                labels = labels + [1]
            else:
                labels = labels + [0]
            odd += 1

            #truncation
            if len(src) == MAX_TOKEN_COUNT:
                break
            elif len(src) > MAX_TOKEN_COUNT:
                src = src[:self.max_token_len - 1] + [src[-1]]
                segs = segs[:self.max_token_len]
                break

        #padding
        if len(src) < MAX_TOKEN_COUNT:
            src = src + [0]*(self.max_token_len - len(src))
            segs = segs + [0]*(self.max_token_len - len(segs))

        if len(clss) < MAX_TOKEN_COUNT:
            clss = clss + [-1]*(self.max_token_len - len(clss))
        if len(labels) < MAX_TOKEN_COUNT:
            labels = labels + [0]*(self.max_token_len - len(labels))

        return dict(
            src = torch.tensor(src),
            segs = torch.tensor(segs),
            clss = torch.tensor(clss),
            labels= torch.FloatTensor(labels)
        )

In [22]:
class SummDataModule(pl.LightningDataModule): # presumm 인코딩 모듈

    def __init__(self, train_df, test_df, val_df, tokenizer, batch_size=1, max_token_len=512):
        super().__init__()
        self.batch_size = batch_size
        self.train_df = train_df
        self.test_df = test_df
        self.val_df = val_df
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len

    def setup(self, stage=None):
        self.train_dataset = SummDataset(
            self.train_df,
            self.tokenizer,
            self.max_token_len
        )

        self.test_dataset = SummDataset(
            self.test_df,
            self.tokenizer,
            self.max_token_len
        )

        self.val_dataset = SummDataset(
            self.val_df,
            self.tokenizer,
            self.max_token_len
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=20 # windows는 0으로 고정해야 에러 안난다. num_workers=2
        )

    def val_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            num_workers=20 # windows는 0으로 고정해야 에러 안난다. num_workers=2
        )

    def test_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            num_workers=20 # windows는 0으로 고정해야 에러 안난다. num_workers=2
        )

In [23]:
data_module = SummDataModule(
  train_df,
  test_df,
  val_df,
  tokenizer,
  batch_size=BATCH_SIZE,
  max_token_len=MAX_TOKEN_COUNT
)

# MODEL
  
kpfBERT를 pretrained_bert로 불러와서 후처리 레이어를 추가하여 문장추출 모델을 만든다.

In [13]:
class PositionalEncoding(nn.Module): # positional embedding

    def __init__(self, dropout, dim, max_len=5000):
        pe = torch.zeros(max_len, dim)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp((torch.arange(0, dim, 2, dtype=torch.float) *
                              -(math.log(10000.0) / dim)))
        pe[:, 0::2] = torch.sin(position.float() * div_term)
        pe[:, 1::2] = torch.cos(position.float() * div_term)
        pe = pe.unsqueeze(0)
        super(PositionalEncoding, self).__init__()
        self.register_buffer('pe', pe)
        self.dropout = nn.Dropout(p=dropout)
        self.dim = dim

    def forward(self, emb, step=None):
        emb = emb * math.sqrt(self.dim)
        if (step):
            emb = emb + self.pe[:, step][:, None, :]

        else:
            emb = emb + self.pe[:, :emb.size(1)]
        emb = self.dropout(emb)
        return emb

    def get_emb(self, emb):
        return self.pe[:, :emb.size(1)]

In [14]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, heads, d_ff, dropout):
        super(TransformerEncoderLayer, self).__init__()

        self.self_attn = MultiHeadedAttention(
            heads, d_model, dropout=dropout)
        self.feed_forward = PositionwiseFeedForward(d_model, d_ff, dropout)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout = nn.Dropout(dropout)

    def forward(self, iter, query, inputs, mask):
        if (iter != 0):
            input_norm = self.layer_norm(inputs)
        else:
            input_norm = inputs

        mask = mask.unsqueeze(1)
        context = self.self_attn(input_norm, input_norm, input_norm,
                                 mask=mask)
        out = self.dropout(context) + inputs
        return self.feed_forward(out)

In [15]:
class ExtTransformerEncoder(nn.Module):
    def __init__(self, hidden_size=768, d_ff=2048, heads=8, dropout=0.2, num_inter_layers=2):
        super(ExtTransformerEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_inter_layers = num_inter_layers
        self.pos_emb = PositionalEncoding(dropout, hidden_size)
        self.transformer_inter = nn.ModuleList(
            [TransformerEncoderLayer(hidden_size, heads, d_ff, dropout)
            for _ in range(num_inter_layers)])
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(hidden_size, eps=1e-6)
        self.wo = nn.Linear(hidden_size, 1, bias=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, top_vecs, mask):
        """ See :obj:`EncoderBase.forward()`"""

        batch_size, n_sents = top_vecs.size(0), top_vecs.size(1)
        pos_emb = self.pos_emb.pe[:, :n_sents]
        x = top_vecs * mask[:, :, None].float()
        x = x + pos_emb

        for i in range(self.num_inter_layers):
            x = self.transformer_inter[i](i, x, x, ~mask)

        x = self.layer_norm(x)
        sent_scores = self.sigmoid(self.wo(x))
        sent_scores = sent_scores.squeeze(-1) * mask.float()

        return sent_scores

In [16]:
class PositionwiseFeedForward(nn.Module):
    """ A two-layer Feed-Forward-Network with residual layer norm.

    Args:
        d_model (int): the size of input for the first-layer of the FFN.
        d_ff (int): the hidden layer size of the second-layer
            of the FNN.
        dropout (float): dropout probability in :math:`[0, 1)`.
    """

    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)

    def gelu(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))


    def forward(self, x):
        inter = self.dropout_1(self.gelu(self.w_1(self.layer_norm(x))))
        output = self.dropout_2(self.w_2(inter))
        return output + x

In [17]:
class MultiHeadedAttention(nn.Module):
    """
    Multi-Head Attention module from
    "Attention is All You Need"
    :cite:`DBLP:journals/corr/VaswaniSPUJGKP17`.

    Similar to standard `dot` attention but uses
    multiple attention distributions simulataneously
    to select relevant items.

    .. mermaid::

       graph BT
          A[key]
          B[value]
          C[query]
          O[output]
          subgraph Attn
            D[Attn 1]
            E[Attn 2]
            F[Attn N]
          end
          A --> D
          C --> D
          A --> E
          C --> E
          A --> F
          C --> F
          D --> O
          E --> O
          F --> O
          B --> O

    Also includes several additional tricks.

    Args:
       head_count (int): number of parallel heads
       model_dim (int): the dimension of keys/values/queries,
           must be divisible by head_count
       dropout (float): dropout parameter
    """

    def __init__(self, head_count, model_dim, dropout=0.1, use_final_linear=True):
        assert model_dim % head_count == 0
        self.dim_per_head = model_dim // head_count
        self.model_dim = model_dim

        super(MultiHeadedAttention, self).__init__()
        self.head_count = head_count

        self.linear_keys = nn.Linear(model_dim,
                                     head_count * self.dim_per_head)
        self.linear_values = nn.Linear(model_dim,
                                       head_count * self.dim_per_head)
        self.linear_query = nn.Linear(model_dim,
                                      head_count * self.dim_per_head)
        self.softmax = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(dropout)
        self.use_final_linear = use_final_linear
        if (self.use_final_linear):
            self.final_linear = nn.Linear(model_dim, model_dim)

    def forward(self, key, value, query, mask=None,
                layer_cache=None, type=None, predefined_graph_1=None):
        """
        Compute the context vector and the attention vectors.

        Args:
           key (`FloatTensor`): set of `key_len`
                key vectors `[batch, key_len, dim]`
           value (`FloatTensor`): set of `key_len`
                value vectors `[batch, key_len, dim]`
           query (`FloatTensor`): set of `query_len`
                 query vectors  `[batch, query_len, dim]`
           mask: binary mask indicating which keys have
                 non-zero attention `[batch, query_len, key_len]`
        Returns:
           (`FloatTensor`, `FloatTensor`) :

           * output context vectors `[batch, query_len, dim]`
           * one of the attention vectors `[batch, query_len, key_len]`
        """

        batch_size = key.size(0)
        dim_per_head = self.dim_per_head
        head_count = self.head_count
        key_len = key.size(1)
        query_len = query.size(1)

        def shape(x):
            """  projection """
            return x.view(batch_size, -1, head_count, dim_per_head) \
                .transpose(1, 2)

        def unshape(x):
            """  compute context """
            return x.transpose(1, 2).contiguous() \
                .view(batch_size, -1, head_count * dim_per_head)

        # 1) Project key, value, and query.
        if layer_cache is not None:
            if type == "self":
                query, key, value = self.linear_query(query), \
                                    self.linear_keys(query), \
                                    self.linear_values(query)

                key = shape(key)
                value = shape(value)

                if layer_cache is not None:
                    device = key.device
                    if layer_cache["self_keys"] is not None:
                        key = torch.cat(
                            (layer_cache["self_keys"].to(device), key),
                            dim=2)
                    if layer_cache["self_values"] is not None:
                        value = torch.cat(
                            (layer_cache["self_values"].to(device), value),
                            dim=2)
                    layer_cache["self_keys"] = key
                    layer_cache["self_values"] = value
            elif type == "context":
                query = self.linear_query(query)
                if layer_cache is not None:
                    if layer_cache["memory_keys"] is None:
                        key, value = self.linear_keys(key), \
                                     self.linear_values(value)
                        key = shape(key)
                        value = shape(value)
                    else:
                        key, value = layer_cache["memory_keys"], \
                                     layer_cache["memory_values"]
                    layer_cache["memory_keys"] = key
                    layer_cache["memory_values"] = value
                else:
                    key, value = self.linear_keys(key), \
                                 self.linear_values(value)
                    key = shape(key)
                    value = shape(value)
        else:
            key = self.linear_keys(key)
            value = self.linear_values(value)
            query = self.linear_query(query)
            key = shape(key)
            value = shape(value)

        query = shape(query)

        key_len = key.size(2)
        query_len = query.size(2)

        # 2) Calculate and scale scores.
        query = query / math.sqrt(dim_per_head)
        scores = torch.matmul(query, key.transpose(2, 3))

        if mask is not None:
            mask = mask.unsqueeze(1).expand_as(scores)
            scores = scores.masked_fill(mask, -1e18) # how can i fix it to use fp16...

        # 3) Apply attention dropout and compute context vectors.

        attn = self.softmax(scores)

        if (not predefined_graph_1 is None):
            attn_masked = attn[:, -1] * predefined_graph_1
            attn_masked = attn_masked / (torch.sum(attn_masked, 2).unsqueeze(2) + 1e-9)

            attn = torch.cat([attn[:, :-1], attn_masked.unsqueeze(1)], 1)

        drop_attn = self.dropout(attn)
        if (self.use_final_linear):
            context = unshape(torch.matmul(drop_attn, value))
            output = self.final_linear(context)
            return output
        else:
            context = torch.matmul(drop_attn, value)
            return context


In [27]:
class Summarizer(pl.LightningModule):  # 요약 모델
    def __init__(self, n_training_steps=None, n_warmup_steps=None):
        super().__init__()
        self.max_pos = 512
        self.bert = BertModel.from_pretrained(BERT_MODEL_NAME, add_pooling_layer=False)
        self.ext_layer = ExtTransformerEncoder()
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.loss = nn.BCELoss(reduction='none')

        # Outputs for train and validation steps
        self.training_step_outputs = []  # 추가
        self.validation_step_outputs = []  # 추가

        for p in self.ext_layer.parameters():
            if p.dim() > 1:
                xavier_uniform_(p)

    def forward(self, src, segs, clss, labels=None):
        # torch 버전에 따라 처리 다름
        mask_src = ~(src == 0)  # 1 - (src == 0)
        mask_cls = ~(clss == -1)  # 1 - (clss == -1)

        top_vec = self.bert(src, token_type_ids=segs, attention_mask=mask_src)
        top_vec = top_vec.last_hidden_state

        sents_vec = top_vec[torch.arange(top_vec.size(0)).unsqueeze(1), clss]
        sents_vec = sents_vec * mask_cls[:, :, None].float()

        sent_scores = self.ext_layer(sents_vec, mask_cls).squeeze(-1)

        loss = 0
        if labels is not None:
            loss = self.loss(sent_scores, labels)
            loss = (loss * mask_cls.float()).sum() / len(labels)

        return loss, sent_scores

    def step(self, batch):
        src = batch['src']
        labels = batch['labels'] if 'labels' in batch and len(batch['labels']) > 0 else None
        segs = batch['segs']
        clss = batch['clss']

        loss, sent_scores = self(src, segs, clss, labels)

        return loss, sent_scores, labels

    def training_step(self, batch, batch_idx):
        loss, sent_scores, labels = self.step(batch)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        self.training_step_outputs.append({'loss': loss, 'predictions': sent_scores, 'labels': labels})
        return {"loss": loss, "predictions": sent_scores, "labels": labels}

    def validation_step(self, batch, batch_idx):
        loss, sent_scores, labels = self.step(batch)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        self.validation_step_outputs.append({'loss': loss, 'predictions': sent_scores, 'labels': labels})
        return {"loss": loss, "predictions": sent_scores, "labels": labels}

    def test_step(self, batch, batch_idx):
        loss, sent_scores, labels = self.step(batch)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        return {"loss": loss, "predictions": sent_scores, "labels": labels}

    def acc_loss(self, outputs):
        if len(outputs) == 0:
            return 0.0, 0.0  # 데이터가 없으면 기본값 반환

        total_loss = 0
        hit_cnt = 0
        for outp in outputs:
            labels = outp['labels'].cpu()
            predictions, idxs = outp['predictions'].cpu().sort()
            loss = outp['loss'].cpu()

            for label, idx in zip(labels, idxs):
                for i in range(1, 3):
                    if label[idx[-i - 1]] == 1:
                        hit_cnt += 1

            total_loss += loss

        avg_loss = total_loss / len(outputs)
        acc = hit_cnt / (3 * len(outputs) * len(labels)) if len(labels) > 0 else 0.0
        return acc, avg_loss

    def on_train_epoch_end(self):
        acc, avg_loss = self.acc_loss(self.training_step_outputs)

        print('acc:', acc, 'avg_loss:', avg_loss)

        self.log('avg_train_loss', avg_loss, prog_bar=True, logger=True)

        # Clear the outputs list for the next epoch
        self.training_step_outputs.clear()

    def on_validation_epoch_end(self):
        acc, avg_loss = self.acc_loss(self.validation_step_outputs)

        print('val_acc:', acc, 'avg_val_loss:', avg_loss)

        self.log('avg_val_loss', avg_loss, prog_bar=True, logger=True)

        # Clear the outputs list for the next epoch
        self.validation_step_outputs.clear()

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=2e-5)

        steps_per_epoch = len(train_df) // BATCH_SIZE
        total_training_steps = steps_per_epoch * N_EPOCHS

        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=steps_per_epoch,
            num_training_steps=total_training_steps
        )

        return dict(
            optimizer=optimizer,
            lr_scheduler=dict(
                scheduler=scheduler,
                interval='step'
            )
        )

In [19]:
model = Summarizer()

You are using a model of type electra to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertModel were not initialized from the model checkpoint at /home/lhch9550/공모전 and are newly initialized: ['encoder.layer.3.attention.self.value.weight', 'encoder.layer.4.attention.self.value.weight', 'encoder.layer.11.attention.self.value.bias', 'encoder.layer.4.attention.output.dense.bias', 'encoder.layer.6.attention.self.query.bias', 'embeddings.word_embeddings.weight', 'encoder.layer.7.attention.output.dense.bias', 'encoder.layer.0.output.dense.bias', 'encoder.layer.7.attention.self.value.weight', 'encoder.layer.3.attention.self.query.weight', 'encoder.layer.9.intermediate.dense.bias', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.2.attention.self.key.bias', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.11.attention.output.LayerNorm.bias', 'enco

# training

In [109]:
#linux
!rm -rf lightning_logs/
!rm -rf checkpoints/

In [22]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="avg_val_loss",
    mode="min"
)

In [23]:
logger = TensorBoardLogger("lightning_logs", name="kpfBERT_Summary")

In [24]:
early_stopping_callback = EarlyStopping(monitor='avg_val_loss', patience=10)

In [25]:
trainer = pl.Trainer(
    logger=logger,
    callbacks=[early_stopping_callback],
    max_epochs=N_EPOCHS,
    accelerator="gpu",  # GPU 사용
    devices=1           # GPU 장치 1개 사용
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, data_module)

In [ ]:
trainer.test(model, data_module)

# predictions

In [ ]:
trained_model = Summarizer.load_from_checkpoint(
    trainer.checkpoint_callback.best_model_path
)
trained_model.eval()
trained_model.freeze()

In [34]:
trainer.checkpoint_callback.best_model_path

''

In [29]:
def data_process(text):
    # 문장 분리 하고,
    sents = kss.split_sentences(text)

    #데이터 가공하고,
    tokenlist = []
    for sent in sents:
        tokenlist.append(tokenizer(
            text = sent,
            add_special_tokens = True)) #, # Add '[CLS]' and '[SEP]'

    src = [] # 토크나이징 된 전체 문단
    labels = []  # 요약문에 해당하면 1, 아니면 0으로 문장수 만큼 생성
    segs = []  #각 토큰에 대해 홀수번째 문장이면 0, 짝수번째 문장이면 1을 매핑
    clss = []  #[CLS]토큰의 포지션값을 지정

    odd = 0

    for tkns in tokenlist:

        if odd > 1 : odd = 0
        clss = clss + [len(src)]
        src = src + tkns['input_ids']
        segs = segs + [odd] * len(tkns['input_ids'])
        odd += 1

        #truncation
        if len(src) == MAX_TOKEN_COUNT:
            break
        elif len(src) > MAX_TOKEN_COUNT:
            src = src[:MAX_TOKEN_COUNT - 1] + [src[-1]]
            segs = segs[:MAX_TOKEN_COUNT]
            break

    #padding
    if len(src) < MAX_TOKEN_COUNT:
        src = src + [0]*(MAX_TOKEN_COUNT - len(src))
        segs = segs + [0]*(MAX_TOKEN_COUNT - len(segs))

    if len(clss) < MAX_TOKEN_COUNT:
        clss = clss + [-1]*(MAX_TOKEN_COUNT - len(clss))

    return dict(
        sents = sents, #정답 출력을 위해...
        src = torch.tensor(src),
        segs = torch.tensor(segs),
        clss = torch.tensor(clss),
    )

In [2]:
def summarize_test(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # GPU 사용 가능 여부 확인
    data = data_process(text.replace('\n', ''))

    # 데이터를 모델과 동일한 장치로 이동
    data['src'] = data['src'].unsqueeze(0).to(device)
    data['segs'] = data['segs'].unsqueeze(0).to(device)
    data['clss'] = data['clss'].unsqueeze(0).to(device)

    # trained_model이 실행 중인 장치 확인 후 모델 이동
    trained_model.to(device)

    # trained_model에 넣어 결과값 반환
    _, rtn = trained_model(data['src'], data['segs'], data['clss'])
    rtn = rtn.squeeze()

    # 예측 결과값을 받기 위한 프로세스
    rtn_sort, idx = rtn.sort(descending=True)

    rtn_sort = rtn_sort.tolist()
    idx = idx.tolist()

    # 예측 결과 중 0까지의 인덱스 찾기
    end_idx = rtn_sort.index(0) if 0 in rtn_sort else len(rtn_sort)

    # 0 이전의 값들만 사용
    rtn_sort = rtn_sort[:end_idx]
    idx = idx[:end_idx]

    # 상위 3개 문장 선택
    rslt = idx[:3] if len(idx) > 5 else idx

    # 최종 요약 결과 리스트 저장
    summ = [data['sents'][r] for r in rslt]  # ✅ 번호 없이 문장만 리스트에 저장

    return summ  # ✅ 요약문 리스트 반환

In [3]:
def summarize_test(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # GPU 사용 가능 여부 확인
    data = data_process(text.replace('\n', ''))

    # 데이터를 모델과 동일한 장치로 이동
    data['src'] = data['src'].unsqueeze(0).to(device)
    data['segs'] = data['segs'].unsqueeze(0).to(device)
    data['clss'] = data['clss'].unsqueeze(0).to(device)

    # trained_model이 실행 중인 장치 확인 후 모델 이동
    trained_model.to(device)

    # trained_model에 넣어 결과값 반환
    _, rtn = trained_model(data['src'], data['segs'], data['clss'])
    rtn = rtn.squeeze()

    # 예측 결과값을 받기 위한 프로세스
    rtn_sort, idx = rtn.sort(descending=True)

    rtn_sort = rtn_sort.tolist()
    idx = idx.tolist()

    # 예측 결과 중 0까지의 인덱스 찾기
    end_idx = rtn_sort.index(0) if 0 in rtn_sort else len(rtn_sort)

    # 0 이전의 값들만 사용
    rtn_sort = rtn_sort[:end_idx]
    idx = idx[:end_idx]

    # 상위 3개 문장 선택
    rslt = idx[:3] if len(idx) > 3 else idx

    # 최종 요약 결과 리스트 저장
    summ = [data['sents'][r] for r in rslt]  

    return " ".join(summ)  

In [30]:
#테스트 문장 입력
test_context = '''이재명 더불어민주당 대선후보는 26일 변호사비 대납 의혹과 관련, "내가 정말로 변호사비를 불법으로 받았으면 나를 구속하라"고 반박했다.
이 후보는 이날 오후 전남 신안군 응급의료 전용헬기 계류장에서 열린 '국민반상회' 후 기자들과 만나 한 시민단체 대표가 고액 수임료 의혹 증거라며 제시한 녹취록에 대해 "조작됐다는 증거를 갖고 있고 검찰에도 제출했다. 검찰과 수사기관들은 빨리 처리하시라"며 이같이 말했다.
앞서 이민구 깨어있는시민연대당 대표는 이 후보가 특정 변호사에게 수임료로 현금과 주식 등 20억원을 줬다는 의혹을 주장하며 녹취록을 제출한 바 있다. 이에 대해 송평수 선대위 부대변인은 "허위사실"이라며 "깨시민당 이 대표에게 제보를 했다는 시민단체 대표 이모 씨가 제3자로부터 기부금을 받아낼 목적으로 허위사실을 녹음한 후, 이 모 변호사에게까지 접근했다. 이러한 비상식적이고 악의적인 행태는 이재명 후보에 대한 정치적 타격을 가할 목적으로 치밀하게 준비한 것"이라고 반박했다.
이에 대해 이 후보는 "그것도 조직폭력배 조작에 버금가는 조작사건이라는 게 곧 드러날 것"이라며 "팩트확인을 하고 언급하면 좋겠다. 당사자도 아니고 제3자들이 자기끼리 녹음한 게 가치가 있느냐"고 반문했다.
그는 "사실이 아니면 무고하고 음해하는 사람들을 무고 혐의나 공직선거법 위반으로 빨리 처리해서 처벌하시라"며 "선거 국면에서 하루이틀도, 한두번도 아니고 '조폭이 뇌물 줬다'는 (허위사실 유포를) 왜 아직도 처리 안 하고 있느냐"고 검경에 불만을 드러냈다.
이어 "허위사실이 드러났으면 당연히 다시는 그런 일이 없게 해야 하는 것 아닌가. 이해가 안 된다"며 "선거관리, 또는 범죄를 단속하는 국가기관들이 이런 식으로 허위사실 유포나 무고 행위를 방치해 정치적 공격 수단으로 쓰게 하면 안 된다"고 했다.
이 후보는 또 자신이 구민주-동교동계와 접촉해 복당을 타진했다는 언론보도와 관련해선 "구체적으로 어떤 사람을 범주별로 나눠 무슨 계, 진영으로 말하는 것은 아니다"라며 "시점을 언젠가 정해 벌점이니, 제재니, 제한이니 다 없애고 모두가 합류할 수 있도록 할 생각"이라고 말했다.
종전에 언급했던 '대사면' 방침을 재확인한 셈이다. 그는 "민주당에 계셨던 분, 또 민주당에 있지 않았더라도 앞으로 함께할 분들에게 계속 연락을 하고 있다"며 "만나고 전화하고 힘을 합치자고 권유하고 있다"고 했다.
그는 " 현재 민주당이 이미 열린민주당과의 통합을 협의하고 있다"며 "거기에 더해서 꼭 민주계라고 말할 필요는 없고 부패사범이나 파렴치범으로 탈당하거나 또는 제명된 사람들이 아니라면, 국가의 미래를 걱정하는 민주개혁 진영의 일원이라면 가리지 말고 과거의 어떤 일이든 그러지 말고 힘을 합치자"고 강조했다.
언론보도에 따르면, 이 후보는 최근 구민주계인 정대철 전 고문과 연락을 주고 받으며 천정배, 정동영 전 의원 등 민주당을 탈당했던 옛 동교동계와 호남 인사들의 복당을 타진했다.
'''

In [31]:
rtn = summarize_test(test_context)
print(rtn)

[Kss]: Oh! You have mecab in your environment. Kss will take this as a backend! :D



NameError: name 'trained_model' is not defined